<a href="https://colab.research.google.com/github/MHaley206265/DS-Unit-1-Sprint-2-Statistics/blob/master/module3/Matt_Haley_DSPT6_123_Introduction_to_Bayesian_Inference_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lambda School Data Science Module 123

## Introduction to Bayesian Inference




## Assignment - Code it up!

We used pure math to apply Bayes Theorem to drug tests. Now write Python code to reproduce the results! This is purposefully open ended - you'll have to think about how you should represent probabilities and events. You can and should look things up.

Specific goals/targets:

### 1) Write a function 

`def prob_drunk_given_positive(prob_drunk_prior, false_positive_rate, true_positive_rate):` 

You should only truly need these two values in order to apply Bayes Theorem. In this example, imagine that individuals are taking a breathalyzer test with an 8% false positive rate, a 100% true positive rate, and that our prior belief about drunk driving in the population is 1/1000. 
 - What is the probability that a person is drunk after one positive breathalyzer test?
 - What is the probability that a person is drunk after two positive breathalyzer tests?
 - How many positive breathalyzer tests are needed in order to have a probability that's greater than 95% that a person is drunk beyond the legal limit?

### 2) Explore `scipy.stats.bayes_mvs`  
Read its documentation, and experiment with it on data you've tested in other ways earlier this week.
 - Create a visualization comparing the results of a Bayesian approach to a traditional/frequentist approach. (with a large sample size they should look close to identical, however, take this opportunity to practice visualizing condfidence intervals in general. The following are some potential ways that you could visualize confidence intervals on your graph:
  - [Matplotlib Error Bars](https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.errorbar.html)
  - [Seaborn barplot with error bars](https://seaborn.pydata.org/generated/seaborn.barplot.html)
  - [Vertical ines to show bounds of confidence interval](https://www.simplypsychology.org/confidence-interval.jpg)
  - [Confidence Intervals on Box Plots](https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.axes.Axes.boxplot.html)

### 3) In your own words, summarize the difference between Bayesian and Frequentist statistics

If you're unsure where to start, check out [this blog post of Bayes theorem with Python](https://dataconomy.com/2015/02/introduction-to-bayes-theorem-with-python/).



In [0]:
import numpy as np
from scipy import stats
import pandas as pd

In [0]:
# 1) prob_drunk function

def prob_drunk_given_positive(prob_drunk_prior, false_positive_rate, true_positive_rate, n):
  complement = 1 - prob_drunk_prior
  true_pos = 1
  marginal = (true_positive_rate * prob_drunk_prior) + (false_positive_rate * complement)
  posterior = 0

  for i in range(n):
    
    posterior = (true_positive_rate * prob_drunk_prior) / marginal
    prob_drunk_prior = posterior
    complement = 1 - prob_drunk_prior
    marginal = (true_positive_rate * prob_drunk_prior) + (false_positive_rate * complement)

  return print(posterior)

In [16]:
prob_drunk_given_positive((1/1000), .08, 1, 10)

0.012357884330202669
0.13525210993291495
0.6615996951348605
0.9606895076105054
0.9967371577896734
0.9997381867081508
0.9999790498904363
0.9999983239589307
0.9999998659165077
0.9999999892733192
0.9999999892733192


In [0]:
# Using the function above, it can be determined that, after one positive
# test, there is only a 1.2% chance that the person is actually drunk.

# After two positive tests, it can be determined that there is a 13.5%
# chance that the person is drunk

# It appears to take 4 positive breathalizer tests to achieve a probability of
# higher than 95%

### Exploring scipy.stats.bayes_mvs

In [0]:
# import a data set

url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data'

column_headers = ['party','handicapped-infants','water-project',
                          'budget','physician-fee-freeze', 'el-salvador-aid',
                          'religious-groups','anti-satellite-ban',
                          'aid-to-contras','mx-missile','immigration',
                          'synfuels', 'education', 'right-to-sue','crime','duty-free',
                          'south-africa']

cvr = pd.read_csv(url,
                  header=None,
                  names=column_headers,
                  na_values="?")

In [19]:
cvr.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,NaN,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,NaN
2,democrat,NaN,y,y,NaN,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,NaN,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,NaN,y,y,y,y


In [21]:
for i in cvr:
  print(cvr[i].value_counts())

democrat      267
republican    168
Name: party, dtype: int64
n    236
y    187
Name: handicapped-infants, dtype: int64
y    195
n    192
Name: water-project, dtype: int64
y    253
n    171
Name: budget, dtype: int64
n    247
y    177
Name: physician-fee-freeze, dtype: int64
y    212
n    208
Name: el-salvador-aid, dtype: int64
y    272
n    152
Name: religious-groups, dtype: int64
y    239
n    182
Name: anti-satellite-ban, dtype: int64
y    242
n    178
Name: aid-to-contras, dtype: int64
y    207
n    206
Name: mx-missile, dtype: int64
y    216
n    212
Name: immigration, dtype: int64
n    264
y    150
Name: synfuels, dtype: int64
n    233
y    171
Name: education, dtype: int64
y    209
n    201
Name: right-to-sue, dtype: int64
y    248
n    170
Name: crime, dtype: int64
n    233
y    174
Name: duty-free, dtype: int64
y    269
n     62
Name: south-africa, dtype: int64


In [23]:
# lets look at mx-missiles

missile_df = cvr[['party', 'mx-missile']].dropna()

missile_df.head()

,party,mx-missile
0,republican,n
1,republican,n
2,democrat,n
3,democrat,n
4,democrat,n


In [24]:
# replace y and n with 1 and 0
missile_df = missile_df.replace({'y':1, 'n':0})

missile_df.head()

,party,mx-missile
0,republican,0
1,republican,0
2,democrat,0
3,democrat,0
4,democrat,0


In [26]:
# find confidence intervals using frequentist and bayesian methods
# Frequentists
freq_conf_int = stats.t.interval(0.95, # This is the alpha value
                                  missile_df.shape[0], # Sample Length
                                  loc = missile_df['mx-missile'].mean(), # mean of data
                                  scale = stats.sem(missile_df['mx-missile']) # std error of mean
                                  )

# Bayesian
bayes_conf_int = stats.bayes_mvs(missile_df['mx-missile'], 0.95)[0][1]

print(f'The frequentists confidence interval is {freq_conf_int}')
print(f'The bayesian confidence interval is {bayes_conf_int}')

The frequentists confidence interval is (0.4527886464902715, 0.5496326610157818)
The bayesian confidence interval is (0.45278830107341983, 0.5496330064326335)


In [36]:
import altair as alt

missile_chart = alt.Chart(missile_df).transform_density(
    'mx-missile',
    as_=['mx-missile', 'density']
).encode(
    x='mx-missile:Q',
    y='density:Q'
)

missile_chart.mark_line()

alt.Chart(...)

## Resources

- [Worked example of Bayes rule calculation](https://en.wikipedia.org/wiki/Bayes'_theorem#Examples) (helpful as it fully breaks out the denominator)
- [Source code for mvsdist in scipy](https://github.com/scipy/scipy/blob/90534919e139d2a81c24bf08341734ff41a3db12/scipy/stats/morestats.py#L139)

## Stretch Goals:

- Go back and study the content from Modules 1 & 2 to make sure that you're really comfortable with them.
- Apply a Bayesian technique to a problem you previously worked (in an assignment or project work) on from a frequentist (standard) perspective
- Check out [PyMC3](https://docs.pymc.io/) (note this goes beyond hypothesis tests into modeling) - read the guides and work through some examples
- Take PyMC3 further - see if you can build something with it!